In [2]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import os
import matplotlib.cm as cm
import datetime 
import time
from scipy.fft import fft, ifft, fftshift
from scipy.fftpack import fftfreq
from EOF import EOF
import gc

In [3]:
import numpy as np
import h5py
from multiprocessing import Pool
from Zonal_Calculation import Zonal_Calculation
import time

# EP flux = - u'v' j + ${f_0 \over N^2} $\partial$ $\theta$ \over $\partial z$

In [ ]:
# # PR = 30 => startfrom 20000 day == final, it doesn't exit xyzt, which is missing
# # Constants
# # PR = 30

# # Saving b and v'b'
# for PR in [0,10,30,50]:
#     print("PR=", PR)
#     days = range(500, 20000, 25)
#     num_days = len(days)
#     num_time_steps = 100  # Assuming each file contributes 100 time steps
#     z_dim, y_dim, x_dim = 20, 64, 128  # Dimensions of z, y, x
    
#     # Pre-allocate the entire array
#     # u_prime_final = np.empty((num_days * num_time_steps, z_dim, y_dim, x_dim), dtype=np.float32)
#     v_prime_final = np.empty((num_days * num_time_steps, z_dim, y_dim, x_dim), dtype=np.float32)
#     theta_final = np.empty((num_days * num_time_steps, z_dim, y_dim, x_dim), dtype=np.float32)
#     theta_prime_final = np.empty((num_days * num_time_steps, z_dim, y_dim, x_dim), dtype=np.float32)
#     b_prime_final = np.empty((num_days * num_time_steps, z_dim, y_dim, x_dim), dtype=np.float32)
    
#     # Initialize Zonal_Calculation object once if all files have the same structure
#     t_zonal_calc  = Zonal_Calculation(3, "grid_t_c_xyzt")
#     p_zonal_calc  = Zonal_Calculation(3, "grid_p_full_xyzt")
#     ps_zonal_calc = Zonal_Calculation(3, "grid_ps_c_xyzt")
#     print("---1---")
    
#     # u_zonal_calc  = Zonal_Calculation(3, "grid_u_c_xyzt")
#     v_zonal_calc  = Zonal_Calculation(3, "grid_v_c_xyzt")
    
    
#     Rd = 287
#     cp = 1004
#     g  = 9.81
#     # Process each day
#     for i, start in enumerate(days):
#         file_path = f"/data92/PeterChang/back_to_master1220/Moist_Dycore/IdealizeSpetral.jl/exp/HSt42/HSt42_{PR}_6hourly_CC_OK/RH80_PR{PR}_20000day_startfrom_{start}day_final.dat"    
#         t_data  = t_zonal_calc.load_data(file_path)
#         p_data  = p_zonal_calc.load_data(file_path)
#         ps_data = ps_zonal_calc.load_data(file_path)
#         # u_data = u_zonal_calc.load_data(file_path)
#         v_data = v_zonal_calc.load_data(file_path)

#         # Cal prime, theta_prime can't be used by class because Dycore dosen't output grid_theta_xyzt
#         # u_prime     = u_zonal_calc.zonal_anomaly(u_data)   # FIXED!!!!!! 8/4 u_prime     = v_zonal_calc.zonal_anomaly(v_data)
#         v_prime     = v_zonal_calc.zonal_anomaly(v_data)  
        
#         theta       = t_data * (ps_data / p_data)**(Rd/cp)
#         theta_prime = theta - theta.mean(axis=3)[:,:,:,np.newaxis] # FIXED!!! Origin:theta_prime = t_zonal_calc.zonal_anomaly(theta) # 
        
#         # Insert data into the pre-allocated array
#         # u_prime_final[i * num_time_steps:(i + 1) * num_time_steps, :, :, :] = u_prime
#         v_prime_final[i * num_time_steps:(i + 1) * num_time_steps, :, :, :] = v_prime
#         theta_final[i * num_time_steps:(i + 1) * num_time_steps, :, :, :]   = theta
#         theta_prime_final[i * num_time_steps:(i + 1) * num_time_steps, :, :, :]   = theta_prime
        
#         # Optional progress logging
#         if start % 1000 == 0:
#             print(f"Processed day {start}")
    
#     print("done")
#     del t_data, ps_data, p_data

#     # Saving
#     # u_prime
#     # u_prime_file = f"/data92/PeterChang/back_to_master1220/Moist_Dycore/IdealizeSpetral.jl/exp/HSt42/6hourly_uv_prime_EMF/PR{PR}/u_prime/PR{PR}_500_20000day_6hourly_u_prime.dat"  
#     # with h5py.File(u_prime_file, 'w') as u_prime_file0:
#     #     u_prime_file0.create_dataset('u_prime',data=u_prime_final)
#     # del u_prime_final
#     # print("---7---")
    
#     # # v_prime
#     # v_prime_file = f"/data92/PeterChang/back_to_master1220/Moist_Dycore/IdealizeSpetral.jl/exp/HSt42/6hourly_uv_prime_EMF/PR{PR}/v_prime/PR{PR}_500_20000day_6hourly_v_prime.dat"  
#     # with h5py.File(v_prime_file, 'w') as v_prime_file0:
#     #     v_prime_file0.create_dataset('v_prime',data=v_prime_final)
#     # del v_prime_final
#     # print("---8---")

#     # # theta
#     # theta_file = f"/data92/PeterChang/back_to_master1220/Moist_Dycore/IdealizeSpetral.jl/exp/HSt42/6hourly_uv_prime_EMF/PR{PR}/theta/PR{PR}_500_20000day_6hourly_theta.dat"  
#     # with h5py.File(theta_file, 'w') as theta_file0:
#     #     theta_file0.create_dataset('theta',data=theta_final)
#     # del theta_final
#     # print("---9---")
    
#     # # theta_prime
#     # theta_prime_file = f"/data92/PeterChang/back_to_master1220/Moist_Dycore/IdealizeSpetral.jl/exp/HSt42/6hourly_uv_prime_EMF/PR{PR}/theta_prime/PR{PR}_500_20000day_6hourly_theta_prime.dat"  
#     # with h5py.File(theta_prime_file, 'w') as theta_prime_file0:
#     #     theta_prime_file0.create_dataset('theta_prime',data=theta_prime_final)
#     # del theta_prime_final
#     # print("---10---")

#     # b_prime
#     b_prime = theta_prime_final / np.mean(theta_final, axis=3)[:,:,:,np.newaxis] * 9.81
#     b_prime_file = f"/data92/PeterChang/back_to_master1220/Moist_Dycore/IdealizeSpetral.jl/exp/HSt42/6hourly_uv_prime_EMF/PR{PR}/b_prime/PR{PR}_500_20000day_6hourly_b_prime.dat"
#     with h5py.File(b_prime_file, 'w') as file:
#         file.create_dataset('b_prime', data=b_prime)
#     del b_prime

#     # Read long time mean theta to calculate b_prime
#     # theta_long_time_file = f"/data92/PeterChang/back_to_master1220/Moist_Dycore/IdealizeSpetral.jl/exp/HSt42/6hourly_uv_prime_EMF/PR{PR}/theta/PR{PR}_500_20000day_6hourly_theta.dat"
#     # theta_long_time_file_read = h5py.File(theta_long_time_file, "r")
#     # theta_long_time = np.asarray(theta_long_time_file_read['theta'][:,:,:,:])
#     # print("---5---")

#     # Read long time mean theta to calculate b_prime
#     # v_prime_file = f"/data92/PeterChang/back_to_master1220/Moist_Dycore/IdealizeSpetral.jl/exp/HSt42/6hourly_uv_prime_EMF/PR{PR}/v_prime/PR{PR}_500_20000day_6hourly_v_prime.dat"
#     # v_prime_file_read = h5py.File(v_prime_file, "r")
#     # v_prime = np.asarray(v_prime_file_read['v_prime'][:5000*4,:,:,:])
#     # print("---5---")
    
#     # v_prime_b_prime
#     vb_mean = np.mean(v_prime * b_prime, axis=(0,3))
#     del theta_prime_final, theta_final
#     print("---6---")

#     # Saving
#     vb_file = f"/data92/PeterChang/back_to_master1220/Moist_Dycore/IdealizeSpetral.jl/exp/HSt42/6hourly_uv_prime_EMF/PR{PR}/v_prime_b_prime/PR{PR}_500_20000day_6hourly_v_prime_b_prime.dat"  
#     with h5py.File(vb_file, 'w') as vb_file0:
#         vb_file0.create_dataset('v_prime_b_prime',data=vb_mean)
#     del vb_mean
#     # print("---5---")
    

    

    



In [4]:
# Original code
def process_pr(PR):
    print("Processing PR=", PR)
    start = time.time()
    days = range(500, 20000, 25)
    num_days = len(days)
    num_time_steps = 100  # Assuming each file contributes 100 time steps
    z_dim, y_dim, x_dim = 20, 64, 128  # Dimensions of z, y, x

    # t_zonal_calc = Zonal_Calculation(3, "grid_t_c_xyzt")
    p_zonal_calc = Zonal_Calculation(3, "grid_p_full_xyzt")
    ps_zonal_calc = Zonal_Calculation(3, "grid_ps_c_xyzt")
    # v_zonal_calc = Zonal_Calculation(3, "grid_v_c_xyzt")
    # u_zonal_calc = Zonal_Calculation(3, "grid_u_c_xyzt")
    print("---Initialized Zonal Calculators---")

    # Memory-mapped arrays for large datasets
    # u_prime_final = np.memmap('/tmp/u_prime_final.dat', dtype=np.float32, mode='w+', shape=(num_days * num_time_steps, z_dim, y_dim, x_dim))
    # v_prime_final = np.memmap('/tmp/v_prime_final.dat', dtype=np.float32, mode='w+', shape=(num_days * num_time_steps, z_dim, y_dim, x_dim))
    # theta_final = np.memmap('/tmp/theta_final.dat', dtype=np.float32, mode='w+', shape=(num_days * num_time_steps, z_dim, y_dim, x_dim))
    # theta_prime_final = np.memmap('/tmp/theta_prime_final.dat', dtype=np.float32, mode='w+', shape=(num_days * num_time_steps, z_dim, y_dim, x_dim))
    # 08/14 add output pressure, surface pressure, and sigma
    p_final = np.memmap('/tmp/p_final.dat', dtype=np.float32, mode='w+', shape=(num_days * num_time_steps, z_dim, y_dim, x_dim))
    ps_final = np.memmap('/tmp/ps_final.dat', dtype=np.float32, mode='w+', shape=(num_days * num_time_steps, z_dim, y_dim, x_dim))
    sigma_final = np.memmap('/tmp/sigma_final.dat', dtype=np.float32, mode='w+', shape=(num_days * num_time_steps, z_dim, y_dim, x_dim))
    # add u_final
    # u_final = np.memmap('/tmp/u_final.dat', dtype=np.float32, mode='w+', shape=(num_days * num_time_steps, z_dim, y_dim, x_dim))

    Rd, cp, g = 287, 1004, 9.81  # Constants

    # Process each day in chunks
    chunk_size = 10  # Adjust chunk size as needed
    for i, start in enumerate(days):
        file_path = f"/data92/PeterChang/back_to_master1220/Moist_Dycore/IdealizeSpetral.jl/exp/HSt42/HSt42_{PR}_6hourly_CC_OK/RH80_PR{PR}_20000day_startfrom_{start}day_final.dat"
        
        # Process data in smaller chunks to reduce memory usage
        for chunk_start in range(0, num_time_steps, chunk_size):
            chunk_end = min(chunk_start + chunk_size, num_time_steps)

            # t_data = t_zonal_calc.load_data(file_path)[chunk_start:chunk_end]
            p_data = p_zonal_calc.load_data(file_path)[chunk_start:chunk_end]
            ps_data = ps_zonal_calc.load_data(file_path)[chunk_start:chunk_end]
            # v_data = v_zonal_calc.load_data(file_path)[chunk_start:chunk_end]
            # u_data = u_zonal_calc.load_data(file_path)[chunk_start:chunk_end]

            # u_prime = u_zonal_calc.zonal_anomaly(u_data)
            # v_prime = v_zonal_calc.zonal_anomaly(v_data)
            # theta = t_data * (ps_data / p_data)**(Rd/cp)
            # theta_prime = theta - np.mean(theta, axis=3, keepdims=True)

            # u_prime_final[i * num_time_steps + chunk_start : i * num_time_steps + chunk_end] = u_prime
            # v_prime_final[i * num_time_steps + chunk_start : i * num_time_steps + chunk_end] = v_prime
            # theta_final[i * num_time_steps + chunk_start : i * num_time_steps + chunk_end] = theta
            # theta_prime_final[i * num_time_steps + chunk_start : i * num_time_steps + chunk_end] = theta_prime
            # 08/14 add output pressure, surface pressure, and sigma
            p_final[i * num_time_steps + chunk_start : i * num_time_steps + chunk_end] = p_data
            ps_final[i * num_time_steps + chunk_start : i * num_time_steps + chunk_end] = ps_data

            # u_final[i * num_time_steps + chunk_start : i * num_time_steps + chunk_end] = u_data

            # Clear memory
            # del u_prime, v_prime, theta, theta_prime
            # del u_data
            
            # 08/14 add
            del p_data, ps_data
            gc.collect()
            
        if start % 1000 == 0:
            print(f"Processed day {start}")
        gc.collect()
        

    # Compute and save b_prime
    # theta_final_zonal_mean = np.mean(theta_final, axis=3, keepdims=True)
    # theta_final_add_dims = np.empty_like(theta_final)
    # theta_final_add_dims[:] = theta_final_zonal_mean

    # b_prime = theta_prime_final / theta_final_add_dims * g
    # Save data using memory-mapped arrays
    common_path = "/data92/PeterChang/back_to_master1220/Moist_Dycore/IdealizeSpetral.jl/exp/HSt42/6hourly_uv_prime_EMF"
    # save_to_hdf5(u_prime_final, "u_prime", common_path + f"/PR{PR}/u_prime/PR{PR}_500_20000day_6hourly_u_prime.dat")
    # save_to_hdf5(v_prime_final, "v_prime", common_path + f"/PR{PR}/v_prime/PR{PR}_500_20000day_6hourly_v_prime.dat")
    # save_to_hdf5(u_prime_final * v_prime_final, "EMF", common_path + f"/PR{PR}/EMF/PR{PR}_500_20000day_6hourly_EMF.dat")
    # save_to_hdf5((u_prime_final * v_prime_final).mean(axis=3), "EMF", common_path + f"/PR{PR}/EMF/PR{PR}_500_20000day_6hourly_EMF_zonal_mean.dat")
    # save_to_hdf5(theta_final, "theta", common_path + f"/PR{PR}/theta/PR{PR}_500_20000day_6hourly_theta.dat")
    # save_to_hdf5(theta_prime_final, "theta_prime", common_path + f"/PR{PR}/theta_prime/PR{PR}_500_20000day_6hourly_theta_prime.dat")

    # # Saving zonal mean var
    # save_to_hdf5(theta_final.mean(axis=3), "theta", common_path + f"/PR{PR}/theta/PR{PR}_500_20000day_6hourly_theta_zonal_mean.dat")
    
    # # Save b_prime and its zonal mean
    # save_to_hdf5(b_prime, "b_prime", common_path + f"/PR{PR}/b_prime/PR{PR}_500_20000day_6hourly_b_prime.dat")
    # save_to_hdf5(b_prime.mean(axis=3), "b_prime", common_path + f"/PR{PR}/b_prime/PR{PR}_500_20000day_6hourly_b_prime_zonal_mean.dat")

    # save_to_hdf5(v_prime_final * b_prime, "v_prime_b_prime", common_path + f"/PR{PR}/v_prime_b_prime/PR{PR}_500_20000day_6hourly_v_prime_b_prime.dat")
    # save_to_hdf5((v_prime_final * b_prime).mean(axis=3), "v_prime_b_prime", common_path + f"/PR{PR}/v_prime_b_prime/PR{PR}_500_20000day_6hourly_v_prime_b_prime_zonal_mean.dat")

    # 08/14 Saving p, ps, and sigma
    sigma = p_final / ps_final
    save_to_hdf5(p_final, "p", common_path + f"/PR{PR}/p/PR{PR}_500_20000day_6hourly_p.dat")
    save_to_hdf5(ps_final, "ps", common_path + f"/PR{PR}/ps/PR{PR}_500_20000day_6hourly_ps.dat")
    save_to_hdf5(sigma, "sigma", common_path + f"/PR{PR}/sigma/PR{PR}_500_20000day_6hourly_sigma.dat")
    save_to_hdf5(sigma.mean(axis=(0,2,3)), "sigma", common_path + f"/PR{PR}/sigma/PR{PR}_500_20000day_6hourly_sigma_only_z.dat")
    # 08/14 add u
    # save_to_hdf5(u_final, "u", common_path + f"/PR{PR}/u/PR{PR}_500_20000day_6hourly_u.dat")
    # save_to_hdf5(u_final.mean(axis=(1,3)), "u", common_path + f"/PR{PR}/u/PR{PR}_500_20000day_6hourly_u_time_y.dat")

    
    
    # Clean up memory-mapped arrays
    # del u_prime_final, v_prime_final, theta_final, theta_prime_final, b_prime
    # 08/14 add
    del p_final, ps_final, sigma
    # del u_final
    gc.collect()
    
    end = time.time()
    print("執行時間：%f 秒" % (end - start))
    print(f"Completed processing and saved b_prime for PR={PR}")

def save_to_hdf5(data, data_name, file_path):
    with h5py.File(file_path, 'w') as file:
        file.create_dataset(data_name, data=data)


if __name__ == "__main__":
    PR_values = [20]
    process_pr(PR_values[0])
    print("All processes completed")

print("Basic variables saving done!")
################################################
# # edit from PR 10 to 50 
# def process_pr(PR):
#     print(f"Processing PR={PR}")
#     start = time.time()
#     days = range(500, 20000, 25)
#     num_days = len(days)
#     num_time_steps = 100  # Assuming each file contributes 100 time steps
#     z_dim, y_dim, x_dim = 20, 64, 128  # Dimensions of z, y, x

#     # Initialize Zonal Calculation (uncomment if needed)
#     # t_zonal_calc = Zonal_Calculation(3, "grid_t_c_xyzt")
#     # p_zonal_calc = Zonal_Calculation(3, "grid_p_full_xyzt")
#     # ps_zonal_calc = Zonal_Calculation(3, "grid_ps_c_xyzt")
#     # v_zonal_calc = Zonal_Calculation(3, "grid_v_c_xyzt")
#     u_zonal_calc = Zonal_Calculation(3, "grid_u_c_xyzt")
#     print("---Initialized Zonal Calculators---")

#     # Memory-mapped array for the u_final dataset
#     u_final = np.memmap('/tmp/u_final.dat', dtype=np.float32, mode='w+', shape=(num_days * num_time_steps, z_dim, y_dim, x_dim))

#     Rd, cp, g = 287, 1004, 9.81  # Constants

#     # Process each day in chunks
#     chunk_size = 10  # Adjust chunk size as needed
#     for i, start in enumerate(days):
#         file_path = f"/data92/PeterChang/back_to_master1220/Moist_Dycore/IdealizeSpetral.jl/exp/HSt42/HSt42_{PR}_6hourly_CC_OK/RH80_PR{PR}_20000day_startfrom_{start}day_final.dat"
        
#         # Process data in smaller chunks to reduce memory usage
#         for chunk_start in range(0, num_time_steps, chunk_size):
#             chunk_end = min(chunk_start + chunk_size, num_time_steps)

#             # Load data
#             u_data = u_zonal_calc.load_data(file_path)[chunk_start:chunk_end]

#             # Save the data in the memory-mapped array
#             u_final[i * num_time_steps + chunk_start : i * num_time_steps + chunk_end] = u_data

#             # Clear memory
#             del u_data
#             gc.collect()
            
#         if start % 1000 == 0:
#             print(f"Processed day {start}")
#         gc.collect()
        
#     # Save data to HDF5
#     common_path = "/data92/PeterChang/back_to_master1220/Moist_Dycore/IdealizeSpetral.jl/exp/HSt42/6hourly_uv_prime_EMF"
#     save_to_hdf5(u_final, "u", common_path + f"/PR{PR}/u/PR{PR}_500_20000day_6hourly_u.dat")
#     save_to_hdf5(u_final.mean(axis=(1,3)), "u", common_path + f"/PR{PR}/u/PR{PR}_500_20000day_6hourly_u_time_y.dat")

#     # Clean up memory-mapped arrays
#     del u_final
#     gc.collect()
    
#     end = time.time()
#     print("Execution time: %f seconds" % (end - start))
#     print(f"Completed processing for PR={PR}")

# def save_to_hdf5(data, data_name, file_path):
#     with h5py.File(file_path, 'w') as file:
#         file.create_dataset(data_name, data=data)

# if __name__ == "__main__":
#     PR_values = [10, 20, 30, 40, 50]
#     for PR in PR_values:
#         process_pr(PR)
#     print("All processes completed")

# print("Basic variables saving done!")


Processing PR= 40
---Initialized Zonal Calculators---
Processed day 1000
Processed day 2000
Processed day 3000
Processed day 4000
Processed day 5000
Processed day 6000
Processed day 7000
Processed day 8000
Processed day 9000
Processed day 10000
Processed day 11000
Processed day 12000
Processed day 13000
Processed day 14000
Processed day 15000
Processed day 16000
Processed day 17000
Processed day 18000
Processed day 19000
執行時間：1723601890.137425 秒
Completed processing and saved b_prime for PR=40
All processes completed
Basic variables saving done!


In [ ]:
# # using ChatGPT
# import h5py
# import numpy as np
# import os
# import gc
# import time

# def process_pr(PR):
#     print("Processing PR=", PR)
#     start_time = time.time()
#     days = range(500, 20000, 25)
#     num_days = len(days)
#     num_time_steps = 100  # Assuming each file contributes 100 time steps
#     z_dim, y_dim, x_dim = 20, 64, 128  # Dimensions of z, y, x

#     t_zonal_calc = Zonal_Calculation(3, "grid_t_c_xyzt")
#     p_zonal_calc = Zonal_Calculation(3, "grid_p_full_xyzt")
#     ps_zonal_calc = Zonal_Calculation(3, "grid_ps_c_xyzt")
#     v_zonal_calc = Zonal_Calculation(3, "grid_v_c_xyzt")
#     u_zonal_calc = Zonal_Calculation(3, "grid_u_c_xyzt")

#     print("---Initialized Zonal Calculators---")

#     # Memory-mapped arrays for large datasets
#     u_prime_final = np.memmap('/tmp/u_prime_final.dat', dtype=np.float32, mode='w+', shape=(num_days * num_time_steps, z_dim, y_dim, x_dim))
#     v_prime_final = np.memmap('/tmp/v_prime_final.dat', dtype=np.float32, mode='w+', shape=(num_days * num_time_steps, z_dim, y_dim, x_dim))
#     theta_final = np.memmap('/tmp/theta_final.dat', dtype=np.float32, mode='w+', shape=(num_days * num_time_steps, z_dim, y_dim, x_dim))
#     theta_prime_final = np.memmap('/tmp/theta_prime_final.dat', dtype=np.float32, mode='w+', shape=(num_days * num_time_steps, z_dim, y_dim, x_dim))

#     Rd, cp, g = 287, 1004, 9.81  # Constants

#     # Process each day in chunks
#     chunk_size = 1500  # Adjust chunk size as needed
#     for i, start_day in enumerate(days):
#         file_path = f"/data92/PeterChang/back_to_master1220/Moist_Dycore/IdealizeSpetral.jl/exp/HSt42/HSt42_{PR}_6hourly_CC_OK/RH80_PR{PR}_20000day_startfrom_{start_day}day_final.dat"
        
#         # Process data in smaller chunks to reduce memory usage
#         for chunk_start in range(0, num_time_steps, chunk_size):
#             chunk_end = min(chunk_start + chunk_size, num_time_steps)

#             t_data = t_zonal_calc.load_data(file_path)[chunk_start:chunk_end]
#             p_data = p_zonal_calc.load_data(file_path)[chunk_start:chunk_end]
#             ps_data = ps_zonal_calc.load_data(file_path)[chunk_start:chunk_end]
#             v_data = v_zonal_calc.load_data(file_path)[chunk_start:chunk_end]
#             u_data = u_zonal_calc.load_data(file_path)[chunk_start:chunk_end]

#             u_prime     = u_zonal_calc.zonal_anomaly(u_data)
#             v_prime     = v_zonal_calc.zonal_anomaly(v_data)
#             theta       = t_data * (ps_data / p_data) ** (Rd / cp)
#             theta_prime = theta - theta.mean(axis=3, keepdims=True)

#             u_prime_final[i * num_time_steps + chunk_start : i * num_time_steps + chunk_end] = u_prime
#             v_prime_final[i * num_time_steps + chunk_start : i * num_time_steps + chunk_end] = v_prime
#             theta_final[i * num_time_steps + chunk_start : i * num_time_steps + chunk_end] = theta
#             theta_prime_final[i * num_time_steps + chunk_start : i * num_time_steps + chunk_end] = theta_prime

#             # Clear memory
#             del u_prime, v_prime, theta, theta_prime
#             gc.collect()

#         if i % 1000 == 0:
#             print(f"Processed up to day {start_day}")
#     gc.collect()

#     # Compute and save b_prime
#     theta_final_zonal_mean = np.mean(theta_final, axis=3, keepdims=True)
#     b_prime = ne.evaluate('theta_prime_final / theta_final_zonal_mean * g')

#     # Save data using memory-mapped arrays
#     common_path = "/data92/PeterChang/back_to_master1220/Moist_Dycore/IdealizeSpetral.jl/exp/HSt42/6hourly_uv_prime_EMF"
#     save_to_hdf5(u_prime_final, "u_prime", common_path + f"/PR{PR}/u_prime/PR{PR}_500_20000day_6hourly_u_prime.dat")
#     save_to_hdf5(v_prime_final, "v_prime", common_path + f"/PR{PR}/v_prime/PR{PR}_500_20000day_6hourly_v_prime.dat")
#     save_to_hdf5(theta_final, "theta", common_path + f"/PR{PR}/theta/PR{PR}_500_20000day_6hourly_theta.dat")
#     save_to_hdf5(theta_prime_final, "theta_prime", common_path + f"/PR{PR}/theta_prime/PR{PR}_500_20000day_6hourly_theta_prime.dat")

#     # Save zonal mean variables
#     save_to_hdf5(theta_final.mean(axis=3), "theta_zonal_mean", common_path + f"/PR{PR}/theta/PR{PR}_500_20000day_6hourly_theta_zonal_mean.dat")
#     save_to_hdf5(b_prime, "b_prime", common_path + f"/PR{PR}/b_prime/PR{PR}_500_20000day_6hourly_b_prime.dat")
#     save_to_hdf5(b_prime.mean(axis=3), "b_prime_zonal_mean", common_path + f"/PR{PR}/b_prime/PR{PR}_500_20000day_6hourly_b_prime_zonal_mean.dat")

#     # Clean up memory-mapped arrays
#     del u_prime_final, v_prime_final, theta_final, theta_prime_final, b_prime
#     gc.collect()

#     end_time = time.time()
#     print(f"Completed processing and saving for PR={PR} in {end_time - start_time:.2f} seconds")

# def save_to_hdf5(data, data_name, file_path):
#     with h5py.File(file_path, 'w') as file:
#         file.create_dataset(data_name, data=data)

# if __name__ == "__main__":
#     PR_values = [40]
#     process_pr(PR_values[0])
#     print("All processes completed")


In [7]:
# # Saving z
# def process_pr(PR):
#     print(f"PR={PR}")
#     days = range(500, 20000, 25)
#     num_days = len(days)
#     num_time_steps = 100  # Assuming each file contributes 100 time steps
#     z_dim, y_dim, x_dim = 20, 64, 128  # Dimensions of z, y, x

#     # Initialize Zonal_Calculation object once if all files have the same structure
#     z_zonal_calc = Zonal_Calculation(3, "grid_z_full_xyzt")
#     # p_zonal_calc = Zonal_Calculation(3, "grid_p_full_xyzt")
#     # ps_zonal_calc = Zonal_Calculation(3, "grid_ps_c_xyzt")

#     # Pre-allocate the entire array
#     # b_prime_final = np.empty((num_days * num_time_steps, z_dim, y_dim, x_dim), dtype=np.float32)
#     z_final = np.empty((num_days * num_time_steps, z_dim, y_dim, x_dim), dtype=np.float32)
#     # theta_prime_final = np.empty((num_days * num_time_steps, z_dim, y_dim, x_dim), dtype=np.float32)

#     Rd = 287
#     cp = 1004

#     # Process each day
#     for i, start in enumerate(days):
#         file_path = f"/data92/PeterChang/back_to_master1220/Moist_Dycore/IdealizeSpetral.jl/exp/HSt42/HSt42_{PR}_6hourly_CC_OK/RH80_PR{PR}_20000day_startfrom_{start}day_final.dat"
#         z_data = z_zonal_calc.load_data(file_path)
#         # p_data = p_zonal_calc.load_data(file_path)
#         # ps_data = ps_zonal_calc.load_data(file_path)

#         # theta = t_data * (ps_data / p_data) ** (Rd / cp)
#         # theta_prime = theta - theta.mean(axis=3)[:, :, :, np.newaxis]
#         z_final[i * num_time_steps:(i + 1) * num_time_steps, :, :, :] = z_data
#         # theta_prime_final[i * num_time_steps:(i + 1) * num_time_steps, :, :, :] = theta_prime

#         if start % 1000 == 0:
#             print(f"Processed day {start}")

#     # b_prime = theta_prime_final / np.mean(theta_final, axis=0) * 9.81
#     # # Save results
#     # b_prime_file = f"/data92/PeterChang/back_to_master1220/Moist_Dycore/IdealizeSpetral.jl/exp/HSt42/6hourly_uv_prime_EMF/PR{PR}/b_prime/PR{PR}_500_20000day_6hourly_b_prime.dat"
#     # with h5py.File(b_prime_file, 'w') as file:
#     #     file.create_dataset('b_prime', data=b_prime)

#     # Save results
#     z_file = f"/data92/PeterChang/back_to_master1220/Moist_Dycore/IdealizeSpetral.jl/exp/HSt42/6hourly_uv_prime_EMF/PR{PR}/z/PR{PR}_500_20000day_6hourly_z.dat"
#     with h5py.File(z_file, 'w') as file:
#         file.create_dataset('dz', data=z_final)
#     print(f"Completed PR={PR}")
#     return f"Completed PR={PR}"


In [8]:
# if __name__ == "__main__":
#     PR_values = [40]
#     with Pool(processes=1) as pool:  # Adjust the number of processes according to your system
#         results = pool.map(process_pr, PR_values)
#     print("All processes completed")
#     # for result in results:
#         # print(result)


PR=20
Processed day 1000
Processed day 2000
Processed day 3000
Processed day 4000
Processed day 5000
Processed day 6000
Processed day 7000
Processed day 8000
Processed day 9000
Processed day 10000
Processed day 11000
Processed day 12000
Processed day 13000
Processed day 14000
Processed day 15000
Processed day 16000
Processed day 17000
Processed day 18000
Processed day 19000
Completed PR=20
All processes completed


In [1]:
# matrix_operation.ipynb

In [ ]:
# import numpy as np
# import matplotlib.pyplot as plt
# from sklearn.decomposition import PCA
# from netCDF4 import Dataset as NetCDFFile
# import random
# import timeit
# import pickle
# from scipy import stats
# import xarray as xr
# import pandas as pd
# import statistics
# from scipy import interpolate
# import os
# import pytz, datetime
# import multiprocessing as mp
# import h5py
# from datetime import date

In [ ]:
# import numpy as np
# import h5py
# from multiprocessing import Pool
# import gc

# # Saving dz
# def process_theta(pr):
#     print(f"Processing PR={pr}")
#     theta_file_path = f"/data92/PeterChang/back_to_master1220/Moist_Dycore/IdealizeSpetral.jl/exp/HSt42/6hourly_uv_prime_EMF/PR{pr}/z/PR{pr}_500_20000day_6hourly_z.dat"
#     with h5py.File(theta_file_path, 'r') as file:
#         theta = np.asarray(file['dz'][:])

#     theta_new = np.einsum('tzyx->txyz', theta)
#     dim_t, dim_z, dim_y, dim_x = theta.shape
#     del theta
#     theta_new = np.reshape(theta_new, [dim_t * dim_x * dim_y, dim_z])

#     L = np.eye(20)
#     central_diff = np.roll(L, -1, axis=0) - np.roll(L, 1, axis=0)
#     backward_diff = L - np.roll(L, -1, axis=1)  # for top boundary

#     theta_central_diff = theta_new.dot(central_diff)
#     theta_backward_diff = theta_new.dot(backward_diff)
#     del theta_new
#     theta_central_diff[:, 0]  = theta_backward_diff[:, 0]   # Use backward diff for first level
#     theta_central_diff[:, 19] = theta_backward_diff[:, 18] # Fix index for last level

#     theta_central_diff = theta_central_diff.reshape([dim_t, dim_x, dim_y, dim_z])
#     theta_central_diff = np.einsum('txyz->tzyx', theta_central_diff)

#     # Calculate mean ignoring time and x dimensions
#     theta_diff_tx_mean = theta_central_diff.mean(axis=(0, 3))
#     theta_central_diff_zonal_mean = theta_central_diff.mean(axis=3)
#     # del theta_central_diff
#     # gc.collect()

#     # posi = np.where(theta_diff_tx_mean[18,:]<0)
#     # theta_diff_tx_mean[18,posi] = theta_diff_tx_mean[17,posi]
#     # posi = np.where(theta_diff_tx_mean[19,:]<0)
#     # theta_diff_tx_mean[19,posi] = theta_diff_tx_mean[18,posi]

#     # Fix negative values based on zonal means
#     for i in range(78000):
#         posi_x, posi_y = np.where(theta_central_diff_zonal_mean[i, :] < 0)
#         theta_central_diff_zonal_mean[i, posi_x, posi_y] = theta_diff_tx_mean[posi_x, posi_y]
#     del theta_diff_tx_mean
#     gc.collect()
#     # Saving
#     dtheta_file = f"/data92/PeterChang/back_to_master1220/Moist_Dycore/IdealizeSpetral.jl/exp/HSt42/6hourly_uv_prime_EMF/PR{pr}/dz/PR{pr}_500_20000day_6hourly_dz_zonal_mean.dat"
#     with h5py.File(dtheta_file, 'w') as file:
#         file.create_dataset('dz', data=theta_central_diff_zonal_mean)
#     del theta_central_diff_zonal_mean

#     dtheta_file = f"/data92/PeterChang/back_to_master1220/Moist_Dycore/IdealizeSpetral.jl/exp/HSt42/6hourly_uv_prime_EMF/PR{pr}/dz/PR{pr}_500_20000day_6hourly_dz.dat"
#     with h5py.File(dtheta_file, 'w') as file:
#         file.create_dataset('dz', data=theta_central_diff)
#     del theta_central_diff

#     gc.collect()    
#     print(f"Completed PR={pr}")
#     # return theta_central_diff_zonal_mean

# if __name__ == "__main__":
#     PR_values = [40]
#     with Pool(processes=1) as pool:
#         pool.map(process_theta, PR_values)
#     print("All processes completed")
#     # for result in results:
#         # print(result)


In [1]:
# import numpy as np
# import h5py
# from multiprocessing import Pool
# import gc

# # Saving dtheta
# def process_theta(pr):
#     print(f"Processing PR={pr}")
#     theta_file_path = f"/data92/PeterChang/back_to_master1220/Moist_Dycore/IdealizeSpetral.jl/exp/HSt42/6hourly_uv_prime_EMF/PR{pr}/theta/PR{pr}_500_20000day_6hourly_theta.dat"
#     with h5py.File(theta_file_path, 'r') as file:
#         theta = np.asarray(file['theta'][:])

#     theta_new = np.einsum('tzyx->txyz', theta)
#     dim_t, dim_z, dim_y, dim_x = theta.shape
#     del theta
#     theta_new = np.reshape(theta_new, [dim_t * dim_x * dim_y, dim_z])

#     L = np.eye(20)
#     central_diff = np.roll(L, -1, axis=0) - np.roll(L, 1, axis=0)
#     backward_diff = L - np.roll(L, -1, axis=1)  # for top boundary

#     theta_central_diff = theta_new.dot(central_diff)
#     theta_backward_diff = theta_new.dot(backward_diff)
#     del theta_new
#     theta_central_diff[:, 0]  = theta_backward_diff[:, 0]   # Use backward diff for first level
#     theta_central_diff[:, 19] = theta_backward_diff[:, 18] # Fix index for last level

#     theta_central_diff = theta_central_diff.reshape([dim_t, dim_x, dim_y, dim_z])
#     theta_central_diff = np.einsum('txyz->tzyx', theta_central_diff)

#     # Calculate mean ignoring time and x dimensions
#     theta_diff_tx_mean = theta_central_diff.mean(axis=(0, 3))
#     theta_central_diff_zonal_mean = theta_central_diff.mean(axis=3)
#     # del theta_central_diff
#     gc.collect()

#     # posi = np.where(theta_diff_tx_mean[18,:]<0)
#     # theta_diff_tx_mean[18,posi] = theta_diff_tx_mean[17,posi]
#     # posi = np.where(theta_diff_tx_mean[19,:]<0)
#     # theta_diff_tx_mean[19,posi] = theta_diff_tx_mean[18,posi]

#     # Fix negative values based on zonal means
#     for i in range(78000):
#         posi_x, posi_y = np.where(theta_central_diff_zonal_mean[i, :] < 0)
#         theta_central_diff_zonal_mean[i, posi_x, posi_y] = theta_diff_tx_mean[posi_x, posi_y]
#     del theta_diff_tx_mean
#     gc.collect()
#     # Saving
#     dtheta_file = f"/data92/PeterChang/back_to_master1220/Moist_Dycore/IdealizeSpetral.jl/exp/HSt42/6hourly_uv_prime_EMF/PR{pr}/dtheta/PR{pr}_500_20000day_6hourly_dtheta_zonal_mean.dat"
#     with h5py.File(dtheta_file, 'w') as file:
#         file.create_dataset('dtheta', data=theta_central_diff_zonal_mean)
#     del theta_central_diff_zonal_mean

#     dtheta_file = f"/data92/PeterChang/back_to_master1220/Moist_Dycore/IdealizeSpetral.jl/exp/HSt42/6hourly_uv_prime_EMF/PR{pr}/dtheta/PR{pr}_500_20000day_6hourly_dtheta.dat"
#     with h5py.File(dtheta_file, 'w') as file:
#         file.create_dataset('dtheta', data=theta_central_diff)
#     del theta_central_diff

#     gc.collect()    
#     print(f"Completed PR={pr}")
#     # return theta_central_diff_zonal_mean

# if __name__ == "__main__":
#     PR_values = [40]
#     with Pool(processes=1) as pool:
#         pool.map(process_theta, PR_values)
#     print("All processes completed")
#     # for result in results:
#         # print(result)


Processing PR=40
Completed PR=40
All processes completed
